In [1]:
import mysql.connector as sql
from tabulate import tabulate
import pandas as pd

In [2]:
connection = sql.connect(
    host="localhost",
    user="root",
    password="Ayoub1111",
    allow_local_infile=True,
    use_pure=True
)

In [3]:
cursor = connection.cursor()

In [4]:
cursor.execute("CREATE DATABASE IF NOT EXISTS product_sales")

In [5]:
cursor.execute('use product_sales')

In [6]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS discount (
        Month VARCHAR(255) NOT NULL,
        Discount_Band VARCHAR(255) NOT NULL,
        Discount INT NOT NULL
);
"""

cursor.execute(create_table_query)

In [7]:
connection.commit()

In [8]:
file_path = '/home/hp/Sql-Sales-Analysis/data/discount_data.csv'
cursor.execute(
    f"""
    LOAD DATA LOCAL INFILE '{file_path}'
    INTO TABLE discount
    FIELDS TERMINATED BY ','
    ENCLOSED BY '"'
    LINES TERMINATED BY '\n'
    IGNORE 1 LINES
    (Month, Discount_Band, Discount)
    """)


In [9]:
connection.commit()

In [10]:
cursor.execute("SHOW TABLES LIKE 'discount'")

In [11]:
result = cursor.fetchone()

In [12]:
if result:
    print("table exists")
else:
    print("table not exists")

table exists


In [13]:
table_name = 'discount'

In [14]:
query = f"SELECT * FROM {table_name}"

In [15]:
cursor.execute(query)
 

In [16]:
result = cursor.fetchall()

In [17]:
product_query = """
    CREATE TABLE IF NOT EXISTS Products (
        ProductID VARCHAR(50) NOT NULL,
        Product VARCHAR(255) NOT NULL,
        Category VARCHAR(255) NOT NULL,
        Cost_Price VARCHAR(255) NOT NULL,
        Sale_Price VARCHAR(255) NOT NULL,
        Brand VARCHAR(255) NOT NULL,
        Description VARCHAR(255) NOT NULL,
        Image_url VARCHAR(255) NOT NULL
);
"""

In [18]:
cursor.execute(product_query)

In [19]:
connection.commit()

In [20]:
file_path = '/home/hp/Sql-Sales-Analysis/data/Product_data.csv'
cursor.execute(
    f"""
    LOAD DATA LOCAL INFILE '{file_path}'
    INTO TABLE Products
    CHARACTER SET utf8mb4
    FIELDS TERMINATED BY ','
    ENCLOSED BY '"'
    LINES TERMINATED BY '\n'
    IGNORE 1 LINES 
    (ProductID, Product, Category, Cost_Price, Sale_Price, Brand, Description, Image_url)
    """)


In [21]:
connection.commit()

In [22]:
# create another tabe 
product_sales_table = """
    CREATE TABLE IF NOT EXISTS ProductSales (
        SalesDate VARCHAR(255) NOT NULL,
        Customer_type VARCHAR(255) NOT NULL,
        Country VARCHAR(255) NOT NULL,
        Product VARCHAR(255) NOT NULL,
        Discount_Band VARCHAR(255) NOT NULL,
        Units_Sold INT NOT NULL
    );
"""
cursor.execute(product_sales_table)

In [23]:
file_path = '/home/hp/Sql-Sales-Analysis/data/product_sales.csv'
cursor.execute(
    f"""
    LOAD DATA LOCAL INFILE '{file_path}'
    INTO TABLE ProductSales
    CHARACTER SET utf8mb4
    FIELDS TERMINATED BY ','
    ENCLOSED BY '"'
    LINES TERMINATED BY '\r\n'
    IGNORE 1 LINES 
    (SalesDate, Customer_type, Country, Product, Discount_Band, Units_Sold)
    """)

In [24]:
connection.commit()

# querying the data 

In [25]:
query1 = "SELECT * FROM ProductSales"
query2= "SELECT * FROM discount"
query3 = "SELECT * FROM Products"

In [26]:
sales_table = pd.read_sql(query1, connection)
discount_table = pd.read_sql(query2, connection)
product_table = pd.read_sql(query3, connection)

/tmp/ipykernel_10668/2324045424.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sales_table = pd.read_sql(query1, connection)
/tmp/ipykernel_10668/2324045424.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  discount_table = pd.read_sql(query2, connection)
/tmp/ipykernel_10668/2324045424.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product_table = pd.read_sql(query3, connection)


In [27]:
sales_table.head()

,SalesDate,Customer_type,Country,Product,Discount_Band,Units_Sold
0,2023-12-01,Government,Germany,SR1001,None,15
1,2023-12-01,Education,United States of America,RR10020,None,6
2,2023-12-01,Government,Canada,FR10021,None,18
3,2023-12-01,Government,Germany,FR10021,None,15
4,2023-12-01,Government,Canada,HY1004,None,18


In [28]:
sales_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3138 entries, 0 to 3137
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   SalesDate      1046 non-null   object
 1   Customer_type  3138 non-null   object
 2   Country        3138 non-null   object
 3   Product        3138 non-null   object
 4   Discount_Band  3138 non-null   object
 5   Units_Sold     3138 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 147.2+ KB


In [29]:
# convert SalesDate column into Date time 
sales_table['SalesDate'] = pd.to_datetime(sales_table['SalesDate'])

In [30]:
sales_table.head()

,SalesDate,Customer_type,Country,Product,Discount_Band,Units_Sold
0,2023-12-01,Government,Germany,SR1001,None,15
1,2023-12-01,Education,United States of America,RR10020,None,6
2,2023-12-01,Government,Canada,FR10021,None,18
3,2023-12-01,Government,Germany,FR10021,None,15
4,2023-12-01,Government,Canada,HY1004,None,18


In [31]:
sales_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3138 entries, 0 to 3137
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   SalesDate      1046 non-null   datetime64[ns]
 1   Customer_type  3138 non-null   object        
 2   Country        3138 non-null   object        
 3   Product        3138 non-null   object        
 4   Discount_Band  3138 non-null   object        
 5   Units_Sold     3138 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 147.2+ KB


In [32]:
product_table.head()
# discount_table.head()

,ProductID,Product,Category,Cost_Price,Sale_Price,Brand,Description,Image_url
0,SR1001,MV7,Dynamic Microphone,$174,$199,Shure,A versatile USB/XLR dynamic microphone ideal f...,https://absentdata.com/wp-content/uploads/2024...
1,RR10020,NT1-A,Condenser Microphone,$160,$229,Rode,A popular condenser microphone known for its l...,https://absentdata.com/wp-content/uploads/2024...
2,FR10021,Scarlett 2i2,Audio Interface,$118,$169,Focusrite,"A compact USB audio interface with two inputs,...",https://absentdata.com/wp-content/uploads/2024...
3,PS10022,AudioBox USB 96 Studio,Recording Bundle,$90,$199,PreSonus,A complete recording bundle that includes an a...,https://absentdata.com/wp-content/uploads/2024...
4,HY1004,QuadCast S,Streaming Microphone,$97,$139,HyperX,A USB condenser microphone featuring customiza...,https://absentdata.com/wp-content/uploads/2024...


In [37]:
sales_table.head()

,SalesDate,Customer_type,Country,Product,Discount_Band,Units_Sold
0,2023-12-01,Government,Germany,SR1001,None,15
1,2023-12-01,Education,United States of America,RR10020,None,6
2,2023-12-01,Government,Canada,FR10021,None,18
3,2023-12-01,Government,Germany,FR10021,None,15
4,2023-12-01,Government,Canada,HY1004,None,18


In [33]:
discount_table.head()

,Month,Discount_Band,Discount
0,January,none,0
1,January,low,6
2,January,medium,10
3,January,high,5
4,February,none,0


In [34]:
discount_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1056 entries, 0 to 1055
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Month          1056 non-null   object
 1   Discount_Band  1056 non-null   object
 2   Discount       1056 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 24.9+ KB
